                                                                                                        Data Processor:                                                                                                                                    
- Zur Verhinderung von Data Leakage: Feature werden generiert die nicht generiert werden können da die Testdaten noch nicht da sind --> sonst ist die Performance in der realen Verwendung schlechter
- Übergabe eines Ordnernamens in dem sich die zu Teilenden Dateien befinden
- Speichert die Dateien in dne gleichen Ordner und benennt sie in ..._train.csv und ..._test.csv

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import os
class DataProcessor:
    def __init__(self, paths="preprocessed", test_size=0.2, random_state=42, stratify_column="group"):
        self.folder = paths
        self.paths = [os.path.relpath(os.path.join(paths, datei)) for datei in os.listdir(paths) if os.path.isfile(os.path.join(paths, datei))]
        print(self.paths)
        self.test_size = test_size
        self.random_state = random_state
        self.stratify_column = stratify_column
        for path in self.paths:
            print(path)
            self.split_and_save(path=path)

    def split_and_save(self, path):
        try:
            test_path = f"{os.path.splitext(path)[0]}_test.csv"
            train_path = f"{os.path.splitext(path)[0]}_train.csv"

            data = pd.read_csv(path, on_bad_lines='skip', sep=';')
            data = data.dropna()
            data = data.drop_duplicates()

            if self.stratify_column not in data.columns:
                raise ValueError(f"Die Spalte '{self.stratify_column}' existiert nicht in der CSV-Datei.")

            X = data.drop(columns=[self.stratify_column])
            y = data[self.stratify_column].values.ravel()

            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
            )

            train_data = X_train.copy()
            train_data[self.stratify_column] = y_train
            train_data.to_csv(train_path, index=False)
            print(f"Trainingsdaten gespeichert unter: {train_path}")

            test_data = X_test.copy()
            test_data[self.stratify_column] = y_test
            test_data.to_csv(test_path, index=False)
            print(f"Testdaten gespeichert unter: {test_path}")

        except FileNotFoundError:
            print(f"Die Datei '{self.path}' wurde nicht gefunden.")
        except ValueError as e:
            print(f"Fehler: {e}")
        except Exception as e:
            print(f"Ein unerwarteter Fehler ist aufgetreten: {e}")